In [ ]:
import torch
import torchvision as tv
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.utils import save_image

num_epochs = 100
batch_size = 64

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
trainset = tv.datasets.MNIST(root='./data',  train=True, download=True, transform=transform)
dataloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder,self).__init__()
        self.encoder = nn.Sequential(
            # 28 x 28
            nn.Conv2d(1, 4, kernel_size=5),
            nn.Dropout2d(p=0.2),
            # 4 x 24 x 24
            nn.ReLU(True),
            nn.Conv2d(4, 8, kernel_size=5),
            nn.Dropout2d(p=0.2),
            # 8 x 20 x 20 = 3200
            nn.ReLU(True),
            nn.Flatten(),
            nn.Linear(3200, 32)
            )
        self.decoder = nn.Sequential(
            nn.Linear(32, 400),
            nn.ReLU(True),
            # 400
            nn.Unflatten(1, (1, 20, 20)),
            # 20 x 20
            nn.Dropout2d(p=0.2),
            nn.ConvTranspose2d(1, 10, kernel_size=5),
            # 24 x 24
            nn.ReLU(True),
            nn.Dropout2d(p=0.2),
            nn.ConvTranspose2d(10, 1, kernel_size=5)
            )
    def forward(self, x):
        out = self.encoder(x)
        out = self.decoder(out)
        return out

model = Autoencoder().to(device)
distance = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        img = img.to(device)
        output = model(img)
        loss = distance(output, img)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('epoch [{}/{}], loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

epoch [1/100], loss: 0.6444
epoch [2/100], loss: 0.5433
epoch [3/100], loss: 0.4430
epoch [4/100], loss: 0.4201
epoch [5/100], loss: 0.4021
epoch [6/100], loss: 0.4328
epoch [7/100], loss: 0.4684
epoch [8/100], loss: 0.3794
epoch [9/100], loss: 0.3339
epoch [10/100], loss: 0.3466
epoch [11/100], loss: 0.3647
epoch [12/100], loss: 0.4495
epoch [13/100], loss: 0.4239
epoch [14/100], loss: 0.3173
epoch [15/100], loss: 0.3306
epoch [16/100], loss: 0.4652
epoch [17/100], loss: 0.4754
epoch [18/100], loss: 0.4985
epoch [19/100], loss: 0.4303
epoch [20/100], loss: 0.3761
epoch [21/100], loss: 0.4724
epoch [22/100], loss: 0.4677
epoch [23/100], loss: 0.4720
epoch [24/100], loss: 0.3533
epoch [25/100], loss: 0.4324


In [4]:
import numpy as np
np.set_printoptions(suppress=True)

confusion_matrix = np.zeros((10, 10))

batch_size = 20*1000

testset = tv.datasets.MNIST(root='./data',  train=False, download=True, transform=transform)
dataloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=4)

for data in dataloader:
    imgs, labels = data
    imgs = Variable(imgs).cpu()
    encs = model.encoder(imgs).detach().numpy()
    for i in range(len(encs)):
        predicted = np.argmax(encs[i])
        actual = labels[i]
        confusion_matrix[actual][predicted] += 1
print(confusion_matrix)
print(int(confusion_matrix.sum()))

[[ 27.   0.  10.   0.   0. 705. 225.  12.   0.   1.]
 [  1.   1.   0.   0.   0.   0. 414.   0. 650.  69.]
 [119.   1.   2.   1.  25.  28. 826.   5.  19.   6.]
 [217.   0.   1.   0.   0. 193. 366.   1.   9. 223.]
 [ 98.  11.  12.   0.   6.   0. 677.   4.   0. 174.]
 [ 49.   0.   2.   0.   0. 131. 591.   3.   1. 115.]
 [118.   0.   0.   0.  95.  24. 678.  40.   1.   2.]
 [ 40.  89. 321.   0.   0.   8. 282.   0.   3. 285.]
 [  8.   0.   0.   0.   0.   9. 919.   0.   0.  38.]
 [ 28.   4.  19.   0.   5.   2. 684.   1.   0. 266.]]
10000


In [5]:
active_neurons = set()
for actual in range(10):
    predicted = np.argmax(confusion_matrix[actual])
    pct = 100 * confusion_matrix[actual][predicted] / np.sum(confusion_matrix[actual])
    print('Actual label %d most likely activates on neuron %d (%d%%)' % (actual, predicted, pct))
    active_neurons.add(predicted)
active_neurons, len(active_neurons)

Actual label 0 most likely activates on neuron 5 (71%)
Actual label 1 most likely activates on neuron 8 (57%)
Actual label 2 most likely activates on neuron 6 (80%)
Actual label 3 most likely activates on neuron 6 (36%)
Actual label 4 most likely activates on neuron 6 (68%)
Actual label 5 most likely activates on neuron 6 (66%)
Actual label 6 most likely activates on neuron 6 (70%)
Actual label 7 most likely activates on neuron 2 (31%)
Actual label 8 most likely activates on neuron 6 (94%)
Actual label 9 most likely activates on neuron 6 (67%)


({2, 5, 6, 8}, 4)